In [2]:
##############################数据读取
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
color=sns.color_palette()
sns.set_style('darkgrid')
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestRegressor ######随机森林
from sklearn.model_selection import train_test_split  #####数据集划分
from sklearn.metrics import mean_absolute_error       ####评价指标MAE(辅助指标)
from sklearn.preprocessing import LabelEncoder       ####编码

from sklearn.linear_model import LinearRegression  #####线性回归
import xgboost as xgb
from sklearn.linear_model import Lasso  ####lasso回归

house_train = pd.read_csv('C:\\Users\\dell\\Desktop\\train_data.csv',encoding = 'gbk')
house_test = pd.read_csv('C:\\Users\\dell\\Desktop\\test_a.csv',encoding = 'gbk')

C:\Users\dell\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\dell\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\dell\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)
C:\Users\dell\Anaconda3\lib\site-packages\pandas\compat\_optional.py:106: UserWarning: Pandas requires version '1.2.1' or newer of 'bottleneck' (version '1.1.0' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
############################数据处理
house_train['pv'].fillna(house_train['pv'].mean(),inplace=True)    #######均值填充
house_train['uv'].fillna(house_train['uv'].mean(),inplace=True)    #######均值填充
house_test['pv'].fillna(house_test['pv'].mean(),inplace=True)
house_test['uv'].fillna(house_test['uv'].mean(),inplace=True)

###################其他变量处理
######将rentType中的'--'改为'未知方式'
house_train.loc[house_train['rentType'] == '--','rentType'] = '未知方式'
house_test.loc[house_train['rentType'] == '--','rentType'] = '未知方式'

######将tradeTime提取交易月/日(利用函数化的写法，可以解决数据类型转换的问题)
def month(x):
    month = int(x.split('/')[1])
    return month
def day(x):
    day = int(x.split('/')[2])
    return day
house_train['month'] = house_train['tradeTime'].apply(lambda x:month(x))
house_train['day'] = house_train['tradeTime'].apply(lambda x:day(x))
house_test['month'] = house_test['tradeTime'].apply(lambda x:month(x))
house_test['day'] = house_test['tradeTime'].apply(lambda x:day(x))

In [4]:
house_train = house_train.loc[(house_train['area'] > 15) & (house_train['area'] < house_train['area'].mean()+3*house_train['area'].std()),:]

In [5]:
##############删除无用特征
house_train.drop('city',axis=1,inplace=True)
house_test.drop('city',axis=1,inplace=True)
house_train.drop('ID',axis=1,inplace=True)
house_test.drop('ID',axis=1,inplace=True)
house_train.drop('tradeTime',axis=1,inplace=True)
house_test.drop('tradeTime',axis=1,inplace=True)

In [6]:
##############buildYear特征修改
house_train.loc[house_train['buildYear'] == '暂无信息','buildYear'] = house_train.loc[house_train['buildYear'] != '暂无信息','buildYear'].median()
house_test.loc[house_test['buildYear'] == '暂无信息','buildYear'] = house_test.loc[house_test['buildYear'] != '暂无信息','buildYear'].median()
house_train['buildYear'] = house_train['buildYear'].astype(int)
house_test['buildYear'] = house_test['buildYear'].astype(int)

In [7]:
#########################################################特征工程
#######################数值型变量特征工程
##########标准化处理
def standard_function(x):
    x_standard = (x-np.mean(x))/(np.std(x))
    return x_standard
house_train.iloc[:,[0,4]] = house_train.iloc[:,[0,4]].apply(lambda x:standard_function(x))
house_test.iloc[:,[0,4]] = house_test.iloc[:,[0,4]].apply(lambda x:standard_function(x))
house_train.iloc[:,10:46] = house_train.iloc[:,10:46].apply(lambda x:standard_function(x))
house_test.iloc[:,10:46] = house_test.iloc[:,10:46].apply(lambda x:standard_function(x))

In [8]:
##########数值型变量基于相关性整合
house_train['pv_uv'] = house_train['pv']+house_train['uv']
house_test['pv_uv'] = house_test['pv']+house_train['uv']

house_train['publicNum'] = 0.125*house_train['hospitalNum']+0.125*house_train['drugStoreNum']+0.125*house_train['gymNum']+0.125*house_train['bankNum']+0.125*house_train['shopNum']+0.125*house_train['parkNum']+0.125*house_train['mallNum']+0.125*house_train['superMarketNum']
house_test['publicNum'] = 0.125*house_test['hospitalNum']+0.125*house_test['drugStoreNum']+0.125*house_test['gymNum']+0.125*house_test['bankNum']+0.125*house_test['shopNum']+0.125*house_test['parkNum']+0.125*house_test['mallNum']+0.125*house_test['superMarketNum']

house_train.drop(['hospitalNum','drugStoreNum','gymNum','bankNum','shopNum','parkNum','mallNum','superMarketNum','totalTradeArea','totalNewTradeArea','supplyLandNum','tradeLandNum','tradeLandNum','landTotalPrice','pv','uv'],axis=1,inplace=True)
house_test.drop('uv',axis=1,inplace=True)


In [9]:
##########数值型变量特征重要性筛选
#####基于与因变量的相关系数
list = []
numeric = ['area','totalFloor','buildYear','saleSecHouseNum','subwayStationNum','busStationNum','interSchoolNum','schoolNum','privateSchoolNum','totalTradeMoney','tradeMeanPrice','tradeSecNum','totalNewTradeMoney','tradeNewMeanPrice','tradeNewNum','remainNewNum','supplyNewNum','supplyLandArea','tradeLandArea','landMeanPrice','totalWorkers','newWorkers','residentPopulation','pv_uv','publicNum']
for i in numeric:
    list.append({i:round((house_train['tradeMoney']).corr(house_train[i]),4)}) 
list

[{'area': 0.0123},
 {'totalFloor': 0.0068},
 {'buildYear': 0.0018},
 {'saleSecHouseNum': -0.0001},
 {'subwayStationNum': -0.0007},
 {'busStationNum': -0.0049},
 {'interSchoolNum': 0.008},
 {'schoolNum': 0.0006},
 {'privateSchoolNum': 0.0069},
 {'totalTradeMoney': 0.0061},
 {'tradeMeanPrice': 0.0071},
 {'tradeSecNum': 0.0048},
 {'totalNewTradeMoney': 0.0098},
 {'tradeNewMeanPrice': 0.0137},
 {'tradeNewNum': -0.0012},
 {'remainNewNum': -0.0019},
 {'supplyNewNum': 0.0009},
 {'supplyLandArea': -0.0028},
 {'tradeLandArea': -0.0021},
 {'landMeanPrice': -0.0018},
 {'totalWorkers': 0.0274},
 {'newWorkers': 0.0818},
 {'residentPopulation': -0.0053},
 {'pv_uv': -0.0038},
 {'publicNum': 0.0037}]

In [10]:
######利用随机森林决定变量的重要性
target = house_train['tradeMoney']   #####经过对数化的y值
train_x_numeric = house_train.loc[:,numeric]   #####数值型x值

from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor()
model.fit(train_x_numeric,target)
list1 = []
for i in range(len(numeric)):
    list1.append({numeric[i]:model.feature_importances_[i]})
list1

[{'area': 0.6365745591854053},
 {'totalFloor': 0.29699760963959687},
 {'buildYear': 0.013479036112433767},
 {'saleSecHouseNum': 0.02211132726979135},
 {'subwayStationNum': 3.11719926626736e-06},
 {'busStationNum': 1.2572694161341492e-05},
 {'interSchoolNum': 5.859954560848767e-06},
 {'schoolNum': 0.0001712565112235613},
 {'privateSchoolNum': 0.0002453676929065552},
 {'totalTradeMoney': 7.266909497721436e-05},
 {'tradeMeanPrice': 0.00035391141641939635},
 {'tradeSecNum': 0.00050269908123949},
 {'totalNewTradeMoney': 0.0012652265063164841},
 {'tradeNewMeanPrice': 0.00011996227107454497},
 {'tradeNewNum': 0.0019557825120210385},
 {'remainNewNum': 0.00041436005855692814},
 {'supplyNewNum': 3.5078995342449913e-06},
 {'supplyLandArea': 1.6134573448434904e-06},
 {'tradeLandArea': 1.765021303898096e-07},
 {'landMeanPrice': 9.938424052326834e-07},
 {'totalWorkers': 8.033203318389406e-06},
 {'newWorkers': 0.02546961891528306},
 {'residentPopulation': 5.12086364449897e-06},
 {'pv_uv': 0.000222373

In [11]:
#######利用线性回归检测特征的重要程度
from sklearn.linear_model import LinearRegression
reg=LinearRegression()
reg.fit(train_x_numeric,target)                         ######线性回归拟合数据
from sklearn.feature_selection import SelectFromModel  #####根据模型结果选择特征
newmodel = SelectFromModel(reg).fit_transform(train_x_numeric,target)
newmodel = pd.DataFrame(newmodel)

In [12]:
#######从原数据中回溯经过selectFromModel筛选后的特征
for i in range(newmodel.shape[1]):
    for j in range(train_x_numeric.shape[1]):
        if newmodel.iloc[0,i] == train_x_numeric.iloc[0,j]:
            newmodel.rename(columns={newmodel.columns[i]:train_x_numeric.columns[j]},inplace=True)
newmodel     

,schoolNum,totalTradeMoney,tradeSecNum,tradeNewNum,newWorkers,residentPopulation,publicNum
0,-0.703352,-0.726870,-0.828056,-0.313751,-0.081075,-0.931804,-0.920244
1,-0.771960,-1.515855,-1.642543,-0.527361,-0.138574,-0.694722,-0.769953
2,-0.188793,2.270107,2.489670,-0.443775,-0.092673,-0.827718,-0.537968
3,0.394374,-0.473265,0.285325,-0.007268,0.048606,-0.201114,0.095267
4,-1.149303,-1.282138,-1.373538,-0.230165,-0.142399,-0.860880,-1.000196
...,...,...,...,...,...,...,...
36833,-0.017273,0.848733,0.165767,0.531400,-0.156835,0.074690,0.009155
36834,0.428678,2.438455,2.168359,-0.545936,-0.156835,-0.673068,0.930562
36835,0.428678,2.918439,2.646590,-0.545936,-0.156835,-0.673068,0.930562
36836,0.428678,-0.311146,-0.050931,-0.490211,-0.156835,0.699341,0.556446


上面从变量相关性、线性回归方法、随机森林特征重要性三种方法进行了特征筛选，结果显示：
1)与租金相关性最高的数值变量是totalworkers、newWorker、area、tradeNewMeanPrice，其他数值变量与因变量相关性普遍不高
2)用随机森林特征选择方法得到的特征重要性中：这4个变量的得分仍较高，另外得分较高的变量还有：totalFloor、buildYear、saleSecHouseNum
3)利用线性回归的特征选择方法选出较为重要的变量有：schoolNum、totalTradeMoney、tradeSecNum、tradeNewNum、newWorkers、residentPopulation、publicNum
综合之后，将上述12个变量带入模型，删除其余的数值型变量。

In [15]:
############分类变量重要性筛选
category = ['rentType','houseType','houseFloor','houseToward','houseDecoration','communityName','region','plate']
train_x_category = house_train.loc[:,category]  ####分类自变量

In [31]:
log_target = np.log(target)   #####将因变量对数化处理,使之符合正态分布
train_x_category = pd.get_dummies(train_x_category) ####分类变量的one-hot处理

#######SelectKBest检验
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
model2 = SelectKBest(f_regression,k='all')
fit = model2.fit(train_x_category,target)
list2 = []
for i in range(train_x_category.shape[1]):
    list2.append({train_x_category.columns[i]:round(fit.scores_[i],3)})
list2

[{'rentType_合租': 0.123},
 {'rentType_整租': 4.063},
 {'rentType_未知方式': 3.159},
 {'houseType_1室0厅0卫': 0.008},
 {'houseType_1室0厅1卫': 0.149},
 {'houseType_1室1厅0卫': 0.001},
 {'houseType_1室1厅1卫': 0.899},
 {'houseType_1室1厅2卫': 0.002},
 {'houseType_1室2厅1卫': 0.046},
 {'houseType_1室2厅2卫': 0.0},
 {'houseType_2室0厅0卫': 0.0},
 {'houseType_2室0厅1卫': 0.023},
 {'houseType_2室0厅2卫': 0.0},
 {'houseType_2室1厅0卫': 0.001},
 {'houseType_2室1厅1卫': 0.015},
 {'houseType_2室1厅2卫': 0.005},
 {'houseType_2室1厅3卫': 0.001},
 {'houseType_2室2厅0卫': 0.0},
 {'houseType_2室2厅1卫': 0.397},
 {'houseType_2室2厅2卫': 0.003},
 {'houseType_2室2厅3卫': 0.009},
 {'houseType_2室3厅1卫': 0.0},
 {'houseType_3室0厅0卫': 0.0},
 {'houseType_3室0厅1卫': 0.001},
 {'houseType_3室0厅2卫': 0.0},
 {'houseType_3室1厅0卫': 0.0},
 {'houseType_3室1厅1卫': 0.149},
 {'houseType_3室1厅2卫': 0.007},
 {'houseType_3室1厅3卫': 0.0},
 {'houseType_3室2厅1卫': 0.071},
 {'houseType_3室2厅2卫': 11.143},
 {'houseType_3室2厅3卫': 0.017},
 {'houseType_3室2厅4卫': 0.001},
 {'houseType_3室2厅5卫': 0.0},
 {'houseType

上面分类变量结果显示：小区名称变量没有重复值，对因变量没有预测意义，其他分类自变量对房屋租金均有不同程度的预测作用。